In [1]:
import os

In [2]:
train_path = '/kaggle/input/train-facial-expresssion/kaggle/working/train'
print(os.listdir(train_path))
test_path = '/kaggle/input/test-facial-expression/kaggle/working/test'
print(os.listdir(test_path))

['surprise', 'fear', 'angry', 'neutral', 'sad', 'disgust', 'happy']
['surprise', 'fear', 'angry', 'neutral', 'sad', 'disgust', 'happy']


### Labelling the images


In [3]:
def load_dataset(dataset_path):
    images = []
    labels = []

    for emotion in os.listdir(dataset_path):
        emotion_path = os.path.join(dataset_path,emotion)
        for image in os.listdir(emotion_path):
            img_path = os.path.join(emotion_path,image)
            images.append(img_path)
            labels.append(emotion)

    return images, labels

In [4]:
import pandas as pd
train_path = '/kaggle/input/train-facial-expresssion/kaggle/working/train'
train_df = pd.DataFrame()
train_df['image'], train_df['label'] = load_dataset(train_path)
train_df.head()

,image,label
0,/kaggle/input/train-facial-expresssion/kaggle/...,surprise
1,/kaggle/input/train-facial-expresssion/kaggle/...,surprise
2,/kaggle/input/train-facial-expresssion/kaggle/...,surprise
3,/kaggle/input/train-facial-expresssion/kaggle/...,surprise
4,/kaggle/input/train-facial-expresssion/kaggle/...,surprise


In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMG_SIZE = (48, 48)  
BATCH_SIZE = 64

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

# Load data using generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="grayscale",
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="grayscale",
    class_mode='categorical',
    shuffle=False
)

# Function to load entire dataset into memory efficiently
def load_data(generator):
    X, y = [], []
    for img_batch, label_batch in generator:
        X.append(img_batch)
        y.append(label_batch)
        if len(X) * BATCH_SIZE >= generator.samples:
            break  # Stop when all images are loaded
    return np.concatenate(X), np.concatenate(y)

# Load train and test data
X_train, y_train = load_data(train_generator)
X_test, y_test = load_data(test_generator)

# Print dataset shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Found 28229 images belonging to 7 classes.
Found 5760 images belonging to 7 classes.
X_train shape: (28229, 48, 48, 1), y_train shape: (28229, 7)
X_test shape: (5760, 48, 48, 1), y_test shape: (5760, 7)


In [13]:
from sklearn.preprocessing import OneHotEncoder
train_df = train_df.sample(frac=1, random_state=0)  
label_column = train_df[['label']]
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(label_column)
column_names = encoder.get_feature_names_out(['label'])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=column_names)
train_encoded = pd.concat([train_df[['image']], one_hot_df], axis=1)
print(train_encoded.iloc[0])


image             /kaggle/input/train-facial-expresssion/kaggle/...
label_angry                                                     0.0
label_disgust                                                   0.0
label_fear                                                      0.0
label_happy                                                     0.0
label_neutral                                                   0.0
label_sad                                                       1.0
label_surprise                                                  0.0
Name: 7648, dtype: object


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(48, 48, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Change to sparse_categorical_crossentropy if labels are integers
              metrics=['accuracy'])

model.summary()  


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 48, 48, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 48, 48, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 24, 24, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 24, 24, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 12, 12, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 6, 6, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 6, 6, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,917,063 (14.94 MB)

 Trainable params: 3,915,143 (14.94 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [16]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Reduce learning rate when validation loss stops improving
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Track validation loss
    patience=3,          # Wait 3 epochs before reducing LR
    factor=0.5,          # Reduce LR by a factor of 0.5
    min_lr=1e-7          # Minimum LR to prevent excessive reduction
)

history = model.fit(
    train_generator,
    epochs=71,  # You can increase epochs if needed
    validation_data=test_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator),
    callbacks=[lr_scheduler]  # Add learning rate scheduler here
)


Epoch 1/71
442/442 ━━━━━━━━━━━━━━━━━━━━ 55s 122ms/step - accuracy: 0.9860 - loss: 0.0429 - val_accuracy: 0.6087 - val_loss: 2.5791 - learning_rate: 1.2500e-04
Epoch 2/71
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 24us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 1.2500e-04
Epoch 3/71
442/442 ━━━━━━━━━━━━━━━━━━━━ 55s 122ms/step - accuracy: 0.9877 - loss: 0.0397 - val_accuracy: 0.6146 - val_loss: 2.7576 - learning_rate: 1.2500e-04
Epoch 4/71
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 24us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 1.2500e-04
Epoch 5/71
442/442 ━━━━━━━━━━━━━━━━━━━━ 52s 117ms/step - accuracy: 0.9889 - loss: 0.0358 - val_accuracy: 0.6161 - val_loss: 2.8544 - learning_rate: 1.2500e-04
Epoch 6/71
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 24us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 1.2500e-04
Epoch 7/71
442/442 ━━━━━━━━━━━━━━━━━━━━ 51s 114ms/step - accuracy: 0.9903 - loss: 0.0302 - val_accuracy: 0.6118 - val_loss: 2.8382 - learning_rate: 1.2500

In [25]:
# Save the new model
model.save("model1.h5")

In [26]:
final_train_acc = history.history["accuracy"][-1]  # Last epoch train accuracy
final_val_acc = history.history["val_accuracy"][-1]  # Last epoch validation accuracy

print(f"Final Training Accuracy: {final_train_acc * 100:.2f}%")
print(f"Final Validation Accuracy: {final_val_acc * 100:.2f}%")

Final Training Accuracy: 99.61%
Final Validation Accuracy: 62.08%
